In [1]:
import websocket
import json

In [2]:
servers = 'localhost:9092'
from kafka import KafkaProducer

In [ ]:
kafka_topic = 'binance_tickers'

In [ ]:
base_url = "wss://stream.binance.com:9443/ws"

top_usdt_pairs = [
    "BTCUSDT", "ETHUSDT", "BNBUSDT", "XRPUSDT", "ADAUSDT",
    "SOLUSDT", "DOGEUSDT", "DOTUSDT", "MATICUSDT", "TRXUSDT",
    "LTCUSDT", "BCHUSDT", "LINKUSDT", "ETCUSDT", "AVAXUSDT",
    "SHIBUSDT", "UNIUSDT", "XLMUSDT", "FILUSDT", "VETUSDT"
]

def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def on_message(ws, message):
    data = json.loads(message)
    print(f"Symbol: {data['s']}, Price: {data['c']}, Volume: {data['v']}")
    
    # Log the message and send to Kafka
    try:
        producer.produce(kafka_topic, key=data['s'], value=json.dumps(data), callback=delivery_report)
        producer.flush()  # Ensure the message is sent before proceeding
    except Exception as e:
        print(f"Failed to send message to Kafka: {e}")

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    # Subscribe to the ticker for each pair
    for pair in top_usdt_pairs:
        ws.send(json.dumps({
            "method": "SUBSCRIBE",
            "params": [f"{pair.lower()}@ticker"],
            "id": 1
        }))

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp(base_url,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()